### 행정안전부 도로명주소 API 활용

In [3]:
import requests, json
import pandas as pd
from urllib.parse import quote

- API Key 가져오기

In [8]:
with open('keys/도로명주소apiKey.txt') as file:
    road_key = file.read()

- URL 만들기

In [5]:
# requests get method를 이용하여 데이터를 보내는 경우에는 한글을 인코딩해야 함
quote('영등포역')

'%EC%98%81%EB%93%B1%ED%8F%AC%EC%97%AD'

In [15]:
base_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
url = f'{base_url}?confmKey={road_key}&currentPage=1&countPerPage=10&keyword={quote("영등포역")}&resultType=json'

response = requests.get(url).text
data = json.loads(response)

data


{'results': {'common': {'errorMessage': '정상',
   'countPerPage': '10',
   'totalCount': '8',
   'errorCode': '0',
   'currentPage': '1'},
  'juso': [{'detBdNmList': '',
    'engAddr': 'B843 Gyeongin-ro, Yeongdeungpo-gu, Seoul',
    'rn': '경인로',
    'emdNm': '영등포동3가',
    'zipNo': '07305',
    'roadAddrPart2': ' (영등포동3가)',
    'emdNo': '03',
    'sggNm': '영등포구',
    'jibunAddr': '서울특별시 영등포구 영등포동3가 33 영등포역지하도상가',
    'siNm': '서울특별시',
    'roadAddrPart1': '서울특별시 영등포구 경인로 지하843',
    'bdNm': '영등포역지하도상가',
    'admCd': '1156010400',
    'udrtYn': '1',
    'lnbrMnnm': '33',
    'roadAddr': '서울특별시 영등포구 경인로 지하843 (영등포동3가)',
    'lnbrSlno': '0',
    'buldMnnm': '843',
    'bdKdcd': '0',
    'liNm': '',
    'rnMgtSn': '115603000028',
    'mtYn': '0',
    'bdMgtSn': '1156010100104230008000001',
    'buldSlno': '0'},
   {'detBdNmList': '',
    'engAddr': '11 Yeongsin-ro 20-gil, Yeongdeungpo-gu, Seoul',
    'rn': '영신로20길',
    'emdNm': '영등포동',
    'zipNo': '07306',
    'roadAddrPart2': ' (영등포동)',
  

In [16]:
base_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f'keyword={quote("영등포역")}&resultType=json'
url = f'{base_url}?{params1}&{params2}'
url

'https://www.juso.go.kr/addrlink/addrLinkApi.do?confmKey=devU01TX0FVVEgyMDIzMDkxNDE0NDMwNjExNDEwMTI=&currentPage=1&countPerPage=10&keyword=%EC%98%81%EB%93%B1%ED%8F%AC%EC%97%AD&resultType=json'

- Open API를 호출하여 결과 받기

In [17]:
result = requests.get(url)
result.status_code

200

In [18]:
result.text[:200]

'{"results":{"common":{"errorMessage":"정상","countPerPage":"10","totalCount":"8","errorCode":"0","currentPage":"1"},"juso":[{"detBdNmList":"","engAddr":"B843 Gyeongin-ro, Yeongdeungpo-gu, Seoul","rn":"경'

In [20]:
res = json.loads(result.text)

- 딕셔너리 데이터에서 필요한 정보 추출

In [22]:
res['results'].keys()

dict_keys(['common', 'juso'])

In [23]:
type(res['results']['juso'])

list

In [24]:
res['results']['juso'][0].keys()

dict_keys(['detBdNmList', 'engAddr', 'rn', 'emdNm', 'zipNo', 'roadAddrPart2', 'emdNo', 'sggNm', 'jibunAddr', 'siNm', 'roadAddrPart1', 'bdNm', 'admCd', 'udrtYn', 'lnbrMnnm', 'roadAddr', 'lnbrSlno', 'buldMnnm', 'bdKdcd', 'liNm', 'rnMgtSn', 'mtYn', 'bdMgtSn', 'buldSlno'])

In [25]:
# 원하는 결과: 도로명 주소
res['results']['juso'][0]['roadAddr']

'서울특별시 영등포구 경인로 지하843 (영등포동3가)'

- 영등포역, 신도림역, 당산역, 여의도역, 영등포구청역

In [27]:
stations = '영등포역 신도림역 당산역 여의도역 영등포구청역'.split()
stations

['영등포역', '신도림역', '당산역', '여의도역', '영등포구청역']

In [33]:
base_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
road_addr_list = []

for station in stations:
    url = f'{base_url}?confmKey={road_key}&currentPage=1&countPerPage=10&keyword={quote(station)}&resultType=json'
    res = requests.get(url).text
    data = json.loads(res)
    road_addr = data['results']['juso'][0]['roadAddr']
    road_addr_list.append(road_addr)

road_addr_list


['서울특별시 영등포구 경인로 지하843 (영등포동3가)',
 '서울특별시 구로구 경인로 625 (신도림동)',
 '서울특별시 영등포구 당산로 229 (당산동6가)',
 '서울특별시 영등포구 여의나루로 지하40 (여의도동)',
 '서울특별시 영등포구 당산로 지하121 (당산동3가)']

In [34]:
df = pd.DataFrame({
    '역명': stations, '주소': road_addr_list
})
df

,역명,주소
0,영등포역,서울특별시 영등포구 경인로 지하843 (영등포동3가)
1,신도림역,서울특별시 구로구 경인로 625 (신도림동)
2,당산역,서울특별시 영등포구 당산로 229 (당산동6가)
3,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동)
4,영등포구청역,서울특별시 영등포구 당산로 지하121 (당산동3가)


In [35]:
df.to_csv('data/영등포소재 지하철역.csv', index=False)
pd.read_csv('data/영등포소재 지하철역.csv')

,역명,주소
0,영등포역,서울특별시 영등포구 경인로 지하843 (영등포동3가)
1,신도림역,서울특별시 구로구 경인로 625 (신도림동)
2,당산역,서울특별시 영등포구 당산로 229 (당산동6가)
3,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동)
4,영등포구청역,서울특별시 영등포구 당산로 지하121 (당산동3가)
